In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [2]:
all_urls = [] #scrapped on 24/03/2021
for i in range(1,35): 
  url = 'https://www.moneycontrol.com/mccode/common/ipo_pagination.php?sec=snapshot3&pgn='+str(i)
  all_urls.append(url)

In [3]:
ipo_list = []
for url in all_urls:
  page  = requests.get(url)
  src = page.content
  soup = BeautifulSoup(src, 'html.parser')
  table_data = soup.find_all('div', attrs = {'class' : 'hist_tbl MT5'})
  column_names = table_data[0].find('table').find_all('th')
  column = []
  for column_name in column_names:
    column.append(str(column_name.text).strip())
  data = table_data[0].find('table').find_all('td')
  ipo = []
  for row in data:
    ipo.append(str(row.text))
  for i in range(0, len(ipo), 8):
    ipo_details = {}
    ipo_details['Equity'] = str(ipo[i+0]).strip()
    ipo_details['Issue price'] = ipo[i+1]
    ipo_details['Listed Date'] = str(ipo[i+2])
    ipo_details['Open'] = ipo[i+3]
    #print(ipo[i+4])
    ipo_details['Close'] = ipo[i+4]
    ipo_details['Listing Gains (%ge)'] = ipo[i+5]
    ipo_details['CMP'] = str(ipo[i+6])
    ipo_details['Current gains %ge'] = ipo[i+7]
    ipo_list.append(ipo_details)

In [4]:
ipo_list[6]

{'CMP': '2,285.90',
 'Close': '3118.65',
 'Current gains %ge': '53.42',
 'Equity': 'Indigo Paints',
 'Issue price': '1490.00',
 'Listed Date': '2nd Feb',
 'Listing Gains (%ge)': '109.31',
 'Open': '2607.50'}

In [5]:
df = pd.DataFrame()
for item in ipo_list:
  df = df.append(item, ignore_index=True)

In [6]:
df

,CMP,Close,Current gains %ge,Equity,Issue price,Listed Date,Listing Gains (%ge),Open
0,502.60,525.90,-9.44,Anupam Rasayan,555.00,24th Mar,-5.24,534.70
1,628.00,812.25,0.16,Heranba,627.00,5th Mar,29.55,900.00
2,128.60,121.40,36.81,Railtel,94.00,26th Feb,29.15,109.00
3,586.80,666.65,46.70,Nureca,400.00,25th Feb,66.66,634.95
4,449.30,445.95,16.70,Stove Kraft,385.00,5th Feb,15.83,498.00
...,...,...,...,...,...,...,...,...
665,596.00,71.10,694.67,Thangamayil,75.00,19th Feb,-5.2,70.00
666,2.25,87.85,-97.00,Syncom Health,75.00,15th Feb,17.13,88.00
667,17.55,146.38,-89.36,Vascon Engineer,165.00,15th Feb,-11.28,155.90
668,"2,849.50",114.50,1865.17,Jubilant Food,145.00,8th Feb,-21.03,160.00


In [7]:
df['CMP'] = pd.to_numeric(df['CMP'], errors='coerce') #for the values which cant be cnverted to float nan will appear that's why using this and not astype('float')
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
df['Current gains %ge'] = pd.to_numeric(df['Current gains %ge'], errors='coerce')
df['Issue price'] = pd.to_numeric(df['Issue price'], errors='coerce')
df['Listing Gains (%ge)'] = pd.to_numeric(df['Listing Gains (%ge)'], errors='coerce')
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')

In [8]:
df.dtypes

CMP                    float64
Close                  float64
Current gains %ge      float64
Equity                  object
Issue price            float64
Listed Date             object
Listing Gains (%ge)    float64
Open                   float64
dtype: object

In [9]:
df.head(5)

,CMP,Close,Current gains %ge,Equity,Issue price,Listed Date,Listing Gains (%ge),Open
0,502.6,525.90,-9.44,Anupam Rasayan,555.0,24th Mar,-5.24,534.70
1,628.0,812.25,0.16,Heranba,627.0,5th Mar,29.55,900.00
2,128.6,121.40,36.81,Railtel,94.0,26th Feb,29.15,109.00
3,586.8,666.65,46.70,Nureca,400.0,25th Feb,66.66,634.95
4,449.3,445.95,16.70,Stove Kraft,385.0,5th Feb,15.83,498.00


In [10]:
column_names = ["Equity", "Issue price", "Listed Date", "Open", "Close", "Listing Gains (%ge)", "CMP", "Current gains %ge"]
df = df.reindex(columns=column_names)

In [11]:
df.tail()

,Equity,Issue price,Listed Date,Open,Close,Listing Gains (%ge),CMP,Current gains %ge
665,Thangamayil,75.0,19th Feb,70.00,71.10,-5.20,596.00,694.67
666,Syncom Health,75.0,15th Feb,88.00,87.85,17.13,2.25,-97.00
667,Vascon Engineer,165.0,15th Feb,155.90,146.38,-11.28,17.55,-89.36
668,Jubilant Food,145.0,8th Feb,160.00,114.50,-21.03,NaN,1865.17
669,Infinite Comp,165.0,3rd Feb,178.35,184.51,11.82,473.75,187.12


In [12]:
df.to_csv('/content/ipo_data.csv') #saving to csv file